---

### 🎓 **Professor**: Apostolos Filippas

### 📘 **Class**: E-Commerce

### 📋 **Topic**: Randomized Assignment and Balance Tests

🚫 **Note**: You are not allowed to share the contents of this notebook with anyone outside this class without written permission by the professor.

---


## Overview

Let's use our Python knowledge to perform a randomized assignment, and verify we did it correctly.

**What we'll learn:**
- How to perform randomized assignment
- How to check if randomization worked
- Balance tests and their importance
- Setting random seeds for reproducibility


In [1]:
# Let's import the libraries we will use
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Load user data for randomization
df_users = pd.read_csv("../data/users.csv")

print("Dataset loaded successfully!")
print(f"Dataset shape: {df_users.shape}")
print(f"Columns: {df_users.columns.tolist()}")

print("Sample of user data:")
print(df_users.head())


Dataset loaded successfully!
Dataset shape: (19272, 9)
Columns: ['user', 'city', 'gender', 'age', 'asset_year', 'user_status', 'earnings', 'first_trip_m', 'price_hourly']
Sample of user data:
   user           city  gender   age  asset_year  user_status  earnings  \
0     1      San Diego    MALE   NaN      2003.0     DELISTED    24.960   
1     2        Oakland    MALE   NaN      2015.0      LIMITED     0.000   
2     3        Chicago    MALE  30.0      2012.0  OFFBOARDING   995.132   
3     4  San Francisco  FEMALE  27.0      2013.0       ACTIVE   235.474   
4     5    Los Angeles    MALE  26.0      2018.0       ACTIVE  1495.042   

   first_trip_m  price_hourly  
0           NaN          8.00  
1          80.0          5.72  
2          41.0          9.50  
3          79.0         11.00  
4          80.0         12.55  


## Exact Randomized Assignment

Sometimes we want exactly half the users in each group. We can achieve this using sampling:


In [2]:
# Exact randomized assignment - pick exactly half
treatment_users = df_users.sample(n=len(df_users) // 2, random_state=44)

df_users["treatment"] = np.where(
    df_users["user"].isin(treatment_users["user"]), "Treatment", "Control"
)

# Check the new assignment counts
df_assignment_exact = (
    df_users.groupby("treatment")
    .agg({"user": "count"})
    .rename(columns={"user": "n"})
    .reset_index()
)

print("Assignment counts (exact split):")
print(df_assignment_exact)


Assignment counts (exact split):
   treatment     n
0    Control  9636
1  Treatment  9636


## Balance Tests

If randomized assignment was performed correctly, then the treatment groups should be similar with respect to the attributes we can observe. Let's test this:


In [3]:
# Gender balance test
df_balance_gender = (
    df_users.groupby(["treatment", "gender"])
    .agg({"user": "count"})
    .rename(columns={"user": "n"})
    .reset_index()
)

print("Gender balance test:")
print(df_balance_gender)

# Calculate percentages within each treatment group
df_balance_gender_pct = (
    df_users.groupby("treatment")["gender"]
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .round(3)
)

print("\nGender balance test (percentages):")
print(df_balance_gender_pct)


Gender balance test:
   treatment  gender     n
0    Control  FEMALE  4363
1    Control    MALE  5267
2  Treatment  FEMALE  4359
3  Treatment    MALE  5274

Gender balance test (percentages):
gender     FEMALE   MALE
treatment               
Control     0.453  0.547
Treatment   0.453  0.547


In [4]:
# Earnings balance test
df_balance_earnings = (
    df_users.groupby("treatment").agg({"earnings": ["mean", "std", "count"]}).round(2)
)

# Flatten column names
df_balance_earnings.columns = ["avg_earnings", "std_earnings", "count"]
df_balance_earnings = df_balance_earnings.reset_index()

print("Earnings balance test:")
print(df_balance_earnings)

# Statistical balance tests
from scipy import stats

# Age balance test (t-test) - filter out users with missing age data
df_users_age = df_users[df_users["age"] > 0]

treatment_ages = df_users_age[df_users_age["treatment"] == "Treatment"]["age"]
control_ages = df_users_age[df_users_age["treatment"] == "Control"]["age"]

age_ttest = stats.ttest_ind(treatment_ages, control_ages)

print(f"\nAge balance t-test:")
print(f"T-statistic: {age_ttest.statistic:.4f}")
print(f"P-value: {age_ttest.pvalue:.4f}")

# Earnings balance test (t-test)
treatment_earnings = df_users[df_users["treatment"] == "Treatment"]["earnings"]
control_earnings = df_users[df_users["treatment"] == "Control"]["earnings"]

earnings_ttest = stats.ttest_ind(treatment_earnings, control_earnings)

print(f"\nEarnings balance t-test:")
print(f"T-statistic: {earnings_ttest.statistic:.4f}")
print(f"P-value: {earnings_ttest.pvalue:.4f}")


Earnings balance test:
   treatment  avg_earnings  std_earnings  count
0    Control        490.47       1058.08   9636
1  Treatment        497.52       1068.95   9636

Age balance t-test:
T-statistic: -0.4601
P-value: 0.6455

Earnings balance t-test:
T-statistic: 0.4596
P-value: 0.6458


In [5]:
# Age balance visualization
plt.figure(figsize=(10, 6))
sns.histplot(data=df_users_age, x="age", hue="treatment", alpha=0.6, bins=30, kde=True)
plt.xlabel("Age")
plt.ylabel("Distribution")
plt.title("Age Balance Test")
plt.legend(title="Treatment Group", loc="upper right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../temp/age_balance_test.pdf", dpi=1000, bbox_inches="tight")
plt.close()

print("Age balance plot saved to temp/age_balance_test.pdf")


Age balance plot saved to temp/age_balance_test.pdf


/var/folders/lw/wwwtpbsj28z8zqcycbpy4t2r0000gn/T/ipykernel_45745/2104037041.py:7: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(title="Treatment Group", loc="upper right")


In [6]:
# Randomization quality checklist
print("\n" + "=" * 60)
print("RANDOMIZATION QUALITY CHECKLIST")
print("=" * 60)

print("\n1. Sample sizes:")
print(df_assignment_exact)

print("\n2. P-values from balance tests (should be > 0.05):")
print(f"   Age balance: {age_ttest.pvalue:.4f}")
print(f"   Earnings balance: {earnings_ttest.pvalue:.4f}")

# For gender balance chi-square test
gender_crosstab = pd.crosstab(df_users["treatment"], df_users["gender"])
chi2, p_value, dof, expected = stats.chi2_contingency(gender_crosstab)
print(f"   Gender balance: {p_value:.4f}")

print(f"\n3. Interpretation:")
print("   P-values > 0.05 indicate good balance")
print("   Visual inspection should show similar distributions")
print("   Large differences in means suggest problems with randomization")



RANDOMIZATION QUALITY CHECKLIST

1. Sample sizes:
   treatment     n
0    Control  9636
1  Treatment  9636

2. P-values from balance tests (should be > 0.05):
   Age balance: 0.6455
   Earnings balance: 0.6458
   Gender balance: 0.9497

3. Interpretation:
   P-values > 0.05 indicate good balance
   Visual inspection should show similar distributions
   Large differences in means suggest problems with randomization


In [7]:
# Set random seed for reproducibility
np.random.seed(42)

# Randomized assignment
# Add a column with random numbers between 0 and 1
df_users["random_number"] = np.random.uniform(0, 1, len(df_users))

# Assign to treatment those users that "drew" more than 0.5
df_users["treatment"] = np.where(df_users["random_number"] > 0.5, "Treatment", "Control")

# Check how many users we assigned to each group
df_assignment = (
    df_users.groupby("treatment")
    .agg({"user": "count"})
    .rename(columns={"user": "n"})
    .reset_index()
)

print("Assignment results:")
print(df_assignment)


Assignment results:
   treatment     n
0    Control  9656
1  Treatment  9616


---

## 🎉 Summary

We learned how to perform randomized assignment and validate it:
- **Random number generation** for assignment (probabilistic and exact)
- **Treatment vs Control** group creation
- **Balance tests** for different variable types (categorical and continuous)
- **Statistical tests** for balance validation (t-tests, chi-square)
- **Visual inspection** of treatment group balance
- **Reproducible results** using random seeds

**Key takeaways:**
- Randomization creates comparable groups
- Balance tests verify randomization quality
- P-values > 0.05 indicate good balance
- Visual inspection complements statistical tests

Randomized assignment is fundamental to experimental design and causal inference.

### Next:
We'll design and analyze controlled experiments

---
